In [1]:
from pydantic import BaseModel

In [4]:
class Person(BaseModel):
    first_name : str
    last_name: str

In [5]:
p = Person(first_name = "test", last_name = "person")

In [6]:
p

Person(first_name='test', last_name='person')

### Deserializing dict to the model 

In [7]:
data = { "first_name" : "test", "last_name" : "person" }

In [9]:
p = Person.model_validate(data)

In [10]:
p

Person(first_name='test', last_name='person')

### Serializing model to dict 

In [11]:
p.model_dump()

{'first_name': 'test', 'last_name': 'person'}

### Serializing model to json 

In [13]:
p.model_dump_json()

'{"first_name":"test","last_name":"person"}'

### Optional fields and Nullable fields

In [24]:
from typing import Union

class Person(BaseModel):
    first_name : str = "test"
    last_name: str
    ethinicity: Union[str,None] = None

In [25]:
p = Person(last_name="person")

In [26]:
p

Person(first_name='test', last_name='person', ethinicity=None)

### Aliases 

In [27]:
from pydantic import Field

class Person(BaseModel):
    first_name : str = Field(alias="firstName",default="test")
    last_name: str
    ethinicity: Union[str,None] = None

In [28]:
data = { "firstName" : "test", "last_name" : "person" }

In [29]:
p = Person.model_validate(data)

In [30]:
p

Person(first_name='test', last_name='person', ethinicity=None)

In [34]:
data = { "first_name" : "test", "last_name" : "person" }

In [35]:
p = Person.model_validate(data)

In [36]:
p

Person(first_name='test', last_name='person', ethinicity=None)

### Default factory (Create default when creating instance) 

In [37]:
from datetime import datetime, timezone

class User(BaseModel):
    created_at: datetime = Field(default_factory= lambda: datetime.now(timezone.utc))

In [38]:
user1 = User()

In [39]:
user1

User(created_at=datetime.datetime(2025, 3, 31, 3, 16, 55, 688378, tzinfo=datetime.timezone.utc))

In [41]:
user1.model_dump()

{'created_at': datetime.datetime(2025, 3, 31, 3, 16, 55, 688378, tzinfo=datetime.timezone.utc)}

In [43]:
user1.model_dump_json() # will get the string representation of the datetime

'{"created_at":"2025-03-31T03:16:55.688378Z"}'

### Custom Serializer

In [46]:
from pydantic import field_serializer

class User(BaseModel):
    created_at: datetime = Field(default_factory= lambda: datetime.now(timezone.utc))
    
    @field_serializer("created_at")
    def serialize_created(self, value):
        return value.strftime("%Y-%m-%d")

In [48]:
user1 = User()
user1.model_dump_json()

'{"created_at":"2025-03-31"}'

### Custom Validator

In [64]:
from pydantic import field_validator

class User(BaseModel):
    age: int
    
    @field_validator("age") # field validator need to be applied to class methods and not instance methods
    def validate_age(cls, value):
        if value < 18:
            raise ValueError("Age cannot be less than 18")

In [69]:
from pydantic import ValidationError

try:
    user1 = User(age=17)
    user1.model_dump_json()
except ValidationError as e:
    print(e)

1 validation error for User
age
  Value error, Age cannot be less than 18 [type=value_error, input_value=17, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


### Custom Validator also transforms the value

In [80]:
from pydantic import field_validator

class User(BaseModel):
    age: int
    
    @field_validator("age") 
    def validate_age(cls, value):
        if value < 18:
            value = -1
            return value

In [82]:
user1 = User(age=17)
user1.model_dump()

{'age': -1}